# utils

> Utility functions useful throughout.

In [2]:
#| default_exp utils

In [3]:
#| hide 
%load_ext autoreload
%autoreload 2

In [4]:
#| export
from pathlib import Path

import numpy as np
import numpy.typing as npt
from typing import Dict, List, Optional, Tuple, Union
from enum import Enum, auto
from typing import ClassVar, List

## CSV Utilities

In [ ]:
#| export

def determine_header_rows_and_delimiter(
    filename: Union[Path, str]
) -> Tuple[Optional[int], Optional[str]]:
    """
    Given a filename pointing at a CSV files, decides:
     * how many header lines there are (based on first line starting with a digit)
     * the delimiter-- right now tries whitespace and comma

    Returns one of:
     - (number of header rows, column delimiter),
     - (number of header rows, None) if the delimiter could not be inferred,
     - (None, None) if CSV has no numerical rows,

    :param filename: CSV Path or filepath literal
    :return: header and delimiter information, if possible.
    """
    MAX_ROWS = 100  # if you don't have data within first 100 lines, exit.
    header_row_count = 0
    with open(filename) as f:
        header_found = False
        line = ""  # modified in while below

        # Search over lines until we find one starting with a digit
        while not header_found:
            line = f.readline()
            if line == "":  # last line of file reached
                return None, None
            line = line.strip()
            try:
                int(line[0])
                header_found = True
            except ValueError:
                header_row_count += 1
            except IndexError:
                header_row_count += 1

            # guard against infinite loop
            if header_row_count >= MAX_ROWS:
                return None, None

        # Now try splitting that first line of data
        delim_guesses = [" ", ",", ", "]

        for guess in delim_guesses:
            try:
                comps = line.split(guess)  # whitespace separated?
                float(comps[0])
                return header_row_count, guess
            except ValueError:
                continue
        return header_row_count, None
